In [49]:
# ! pip install findspark

In [48]:
# pip install pyspark==3.0.3

In [47]:
# !hadoop distcp \
#         -D fs.s3a.bucket.mlops-backet-16102023.endpoint=storage.yandexcloud.net \
#         -D fs.s3a.bucket.mlops-backet-16102023.acces.key=YCAJEvkQyzISgNsKNE_LOpEQi \
#         -D fs.s3a.bucket.mlops-backet-16102023.secret.key=YCObhqtjSfTFzQnSwBOxlHcrX6Q-V6Lr-WMMnvVc \
#         -update -skipcrccheck -numListstatusThreads 10 \
#         s3a://mlops-backet-16102023/2019-09-21.txt \
#     hdfs://rc1a-dataproc-m-quy5gqnsledmjm7d.mdb.yandexcloud.net/user/root/datasets/set01/

В данном notebook представлено описание найденных ошибок. В конце представлен скрипт для очистки данных.
В бакет залил 3 проанализированных дата сета + краткую информацию по датасетам</br>
Основные ошибки, некорректные данные:</br>
1. значения стобцов пропущены, null, nan</br>
2. столбец tx_amount принимает по операциям знаечние "0"</br>
3. стобец terminal_id > 999, в самом первом датасете 2019-08-22.txt был в диапазоне 0-999. в файле 2019-12-20.txt нашел выбросы > 999. 15290 строк таких в файле 2019-12-20.txt</br>
4. повторяющиеся значения

In [1]:
import findspark
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as F

In [2]:
spark = (
    SparkSession
        .builder
        .appName("mlop-validation-dataset")
        .config("spark.dynamicAllocation.enabled", "true")
        .config("spark.executor.memory", "4g")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
)

In [8]:
# чтение txt файла и преобразование в df
file_dir = "/user/root/datasets/fraudset/2019-09-21.txt"
import re
from pyspark.sql.types import FloatType

rdd = spark.sparkContext.textFile("{0}".format(file_dir))
header = rdd.take(1)[0]
df = rdd.filter(lambda r: r != header) \
        .map(lambda r: r.split(',')) \
        .toDF(re.sub(r'\s+', '', header).split('|'))

df = df.withColumnRenamed('#tranaction_id', 'tranaction_id')

df = df.withColumn('tx_amount', df['tx_amount'].cast(FloatType()))

print(df.show())

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|     46988237|2019-09-21 09:45:59|          1|        178|    83.11|        2627159|          30|       0|                0|
|     46988238|2019-09-21 19:33:01|          2|        660|    22.15|        2662381|          30|       0|                0|
|     46988239|2019-09-21 18:06:19|          3|        732|    36.83|        2657179|          30|       0|                0|
|     46988240|2019-09-21 16:56:01|         10|        663|     19.3|        2652961|          30|       0|                0|
|     46988241|2019-09-21 05:34:26|         10|        145|   106.51|        2612066|          30|       0|           

In [9]:
# чтение txt файла и преобразование в df
import re
from pyspark.sql.types import FloatType

rdd = spark.sparkContext.textFile("/user/root/datasets/fraudset/2019-08-22.txt")
header = rdd.take(1)[0]
df = rdd.filter(lambda r: r != header) \
        .map(lambda r: r.split(',')) \
        .toDF(re.sub(r'\s+', '', header).split('|'))

df = df.withColumnRenamed('#tranaction_id', 'tranaction_id')

df = df.withColumn('tx_amount', df['tx_amount'].cast(FloatType()))

print(df.show())

+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|tranaction_id|        tx_datetime|customer_id|terminal_id|tx_amount|tx_time_seconds|tx_time_days|tx_fraud|tx_fraud_scenario|
+-------------+-------------------+-----------+-----------+---------+---------------+------------+--------+-----------------+
|            0|2019-08-22 06:51:03|          0|        711|    70.91|          24663|           0|       0|                0|
|            1|2019-08-22 05:10:37|          0|          0|    90.55|          18637|           0|       0|                0|
|            2|2019-08-22 19:05:33|          0|        753|    35.38|          68733|           0|       0|                0|
|            3|2019-08-22 07:21:33|          0|          0|    80.41|          26493|           0|       0|                0|
|            4|2019-08-22 09:06:17|          1|        981|   102.83|          32777|           0|       0|           

In [4]:
df.printSchema() #схема данных

root
 |-- tranaction_id: string (nullable = true)
 |-- tx_datetime: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- terminal_id: string (nullable = true)
 |-- tx_amount: float (nullable = true)
 |-- tx_time_seconds: string (nullable = true)
 |-- tx_time_days: string (nullable = true)
 |-- tx_fraud: string (nullable = true)
 |-- tx_fraud_scenario: string (nullable = true)



Общий анализ DF, информация по кол-ву строк, мошенническим операциям,легитимным операциям, количеству уникальных и повторяющихся строк

In [5]:
%%time
len_df = df.count() #количество всех строк
len_fraud_df = df.filter("tx_fraud = 1").count()
len_without_fraud = len_df - len_fraud_df
len_distinct_rows = df.distinct().count()
print('Всего строк: ', len_df)
print('Количество повторяющихся строк: ', len_df - len_distinct_rows)
print('Количество мошеннических операций: ', len_fraud_df)
print('Количество легитимных операций: ', len_without_fraud)

Всего строк:  46988418
Количество повторяющихся строк:  181
Количество мошеннических операций:  2527005
Количество легитимных операций:  44461413
CPU times: user 21.2 ms, sys: 26.4 ms, total: 47.7 ms
Wall time: 3min 42s


Строки равные нулю, None, Null по столбцам tx_datetime, customer_id, customer_id.</br>
По столбцу tx_amount равны 0

In [6]:
%%time
from pyspark.sql.functions import col, asc, desc
import numpy as np
import pandas as pd
import re
from pyspark.sql.types import FloatType

string_columns = ['tx_datetime', 'customer_id', 'terminal_id']
numeric_columns = ['tx_amount']
missing_values = {}
for index, column in enumerate(df.columns):
    if column in string_columns:
        missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull() | (col(column) == 0)).count()
        missing_values.update({column:missing_count})
    if column in numeric_columns:
        missing_count = df.filter((col(column) == 0)).count()
        missing_values.update({column:missing_count})
        
missing_df = pd.DataFrame.from_dict([missing_values])
missing_df

CPU times: user 21.5 ms, sys: 15.6 ms, total: 37.1 ms
Wall time: 3min 52s


,tx_datetime,customer_id,terminal_id,tx_amount
0,0,66,2041671,884


Количество строк со значением Null, пропусками в датасете

In [10]:
from pyspark.sql.functions import col, isnan, when, trim

def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


count_null_rows = df.select([to_null(c).alias(c) for c in df.columns]).na.drop().count()
print('Количество строк с пропущенными данными: ', len_df - count_null_rows)

Количество пропущенных строк:  0


Очистка датасета, заметил в конце ошибку, я не исключил terminal_id = 0. В скрипте исправления внес

In [11]:
from pyspark.sql.functions import col, isnan, when, trim

df_validation = (
    df
    .filter(col("terminal_id") <= 999)
    .filter(col("tx_amount") > 0)
    .filter(col("customer_id") > 0)
    .distinct()
)


# пропуски и значения Null
def to_null_bool(c, dt):
    if df == "float":
        return c.isNull() | isnan(c)
    elif df == "string":
        return ~c.isNull() & (trim(c) != "")
    else:
        return ~c.isNull()

# Only keep columns with not empty strings
def to_null(c, dt):
    c = col(c)
    return when(to_null_bool(c, dt), c)


df_validation = df_validation.select([to_null(c, dt[1]).alias(c) for c, dt in zip(df_validation.columns, df_validation.dtypes)]).na.drop(how="any")

len_val_df = df_validation.count()

print('Строк удалено :', len_df - len_val_df)

Строк удалено : 16856


Запись датасета в формат паркета

In [ ]:
file_dir = "/user/root/datasets/set02/2019-08-22.txt"
# file_name = file_dir.split("/")[len(file_name) - 1]
file_name = file_dir.split("/")[len(file_dir.split("/")) - 1]

In [28]:
df_validation.write.parquet("/user/root/datasets/fraudset_parq/df_validation_{0}.parquet".format(file_name[:-4]))

In [44]:
spark.stop()

Итоговый скрипт

In [46]:
#создать файл month_stat.py
#запустить приложение /usr/bin/spark-submit month_stat.py

import findspark
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
import pyspark.sql.functions as F
import re
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, asc, desc
import numpy as np
import pandas as pd
import re
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, isnan, when, trim




def write_info(info, name_file):
    with open(r"{0}".format(name_file), "w") as file:
        for k, v in info.items():
            file.write('{0} {1} \n'.format(k, v))
            
            
def to_null(c):
    return when(~(col(c).isNull() | isnan(col(c)) | (trim(col(c)) == "")), col(c))


def main():
    spark = (
        SparkSession
            .builder
            .appName("mlop-validation-dataset")
            .config("spark.dynamicAllocation.enabled", "true")
            .config("spark.executor.memory", "4g")
            .config("spark.driver.memory", "4g")
            .getOrCreate()
    )
    file_dir = "/user/root/datasets/fraudset/2022-06-07.txt" #задал явно, потому нужно будет через *.txt?
    file_name = file_dir.split("/")[len(file_dir.split("/")) - 1]

    rdd = spark.sparkContext.textFile("{0}".format(file_dir))
    header = rdd.take(1)[0]
    df = rdd.filter(lambda r: r != header) \
            .map(lambda r: r.split(',')) \
            .toDF(re.sub(r'\s+', '', header).split('|'))

    df = df.withColumnRenamed('#tranaction_id', 'tranaction_id')

    df = df.withColumn('tx_amount', df['tx_amount'].cast(FloatType()))
    
    
    len_df = df.count() #количество всех строк
    len_fraud_df = df.filter("tx_fraud = 1").count()
    len_without_fraud = len_df - len_fraud_df
    len_distinct_rows = df.distinct().count()
    
    df_info = {'count_all_rows: ': len_df,
               'count_duplicates_rows: ': len_df - len_distinct_rows,
               'count_fraud_rows: ': len_fraud_df,
               'count_without_fraud_rows: ': len_without_fraud}
                
    string_columns = ['tx_datetime', 'customer_id', 'terminal_id']
    numeric_columns = ['tx_amount']
        
    for index, column in enumerate(df.columns):
        if column in string_columns:
            missing_count = df.filter(col(column).eqNullSafe(None) | col(column).isNull() | (col(column) == 0)).count()
            df_info.update({"missings_{0}".format(column):missing_count})
        if column in numeric_columns:
            missing_count = df.filter((col(column) == 0)).count()
            df_info.update({"missings_{0}".format(column):missing_count})
            
    negative_values = df.filter('tx_amount < 0').count()
    df_info.update({"negative_values_tx_amount":negative_values})
    
    
    df_validation = (
        df
            .filter(col("terminal_id").between('1', '999'))
            .filter(col("tx_amount") > 0)
            .filter(col("customer_id") > 0)
            .distinct()
    )
    
    df_validation = df_validation.select([to_null(c).alias(c) for c in df.columns]).na.drop()
    len_val_df = df_validation.count()
    
    df_info.update({"count_del_rows":len_df - len_val_df})
    
    write_info(df_info, "{0}_info.txt".format(file_name))
    
    df_validation.write.parquet("/user/root/datasets/fraudset_parq/df_validation_{0}.parquet".format(file_name[:-4]))
    
    
    spark.stop()

if __name__ == "__main__":
        main()

In [1]:
# !hdfs dfs -ls /user/root/datasets/set01/

-rw-r--r--   1 ubuntu hadoop 2854479008 2023-11-05 09:09 /user/root/datasets/set01


In [ ]:
#пробую удалить из DF терминалы с 0% мошенничества

In [40]:
%%time

df_only_fraud_terminal_id = (
    df_validation
    .select("terminal_id", "tx_fraud")
    .filter("tx_fraud = 1")
    .groupBy("terminal_id")
    .count()
    .withColumnRenamed("count","tx_fraud_1_terminal")
)

CPU times: user 3.82 ms, sys: 0 ns, total: 3.82 ms
Wall time: 43.3 ms


In [41]:
%%time

df_all_transaction_terminal_id = (
    df_validation
    .select("terminal_id")
    .groupBy("terminal_id")
    .count()
    .withColumnRenamed("count","terminal_cnt_tranaction_id_for")
)

CPU times: user 674 µs, sys: 2.59 ms, total: 3.26 ms
Wall time: 27.4 ms


In [43]:
%%time
(
    df_all_transaction_terminal_id
    .join(df_only_fraud_terminal_id, on='terminal_id', how="left")
    .filter("tx_fraud_1_terminal = 0")
    .limit(10)
    .show()
    
)

+-----------+------------------------------+-------------------+
|terminal_id|terminal_cnt_tranaction_id_for|tx_fraud_1_terminal|
+-----------+------------------------------+-------------------+
+-----------+------------------------------+-------------------+

CPU times: user 56.6 ms, sys: 26.3 ms, total: 83 ms
Wall time: 2min 46s
